## Load model

In [1]:
import os
from utils_lib import utils
from utils_lib.etdata import ETData
from model import gazeNET as gazeNET
import model as model_func

logdir = os.path.join('logdir', 'model_final')
fname_config = os.path.join(logdir, 'config.json')
configuration = utils.Config(fname_config)
config = configuration.params

model_name = '%s.pth.tar' % 'gazeNET_0004_00003750'

num_classes = len(config['events'])
model = gazeNET(config, num_classes)
model_func.load(model, 'model_final', config, model_name)
model.eval()


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


logdir/model_final/models/gazeNET_0004_00003750.pth.tar
Loading model: logdir/model_final/models/gazeNET_0004_00003750.pth.tar
done.


gazeNET(
  (conv_stack): Sequential(
    (conv1): Sequential(
      (0): Conv2d(1, 8, kernel_size=(2, 11), stride=(1, 1), padding=(1, 5), bias=False)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (conv2): Sequential(
      (0): Conv2d(8, 8, kernel_size=(2, 11), stride=(1, 1), padding=(1, 5), bias=False)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace=True)
      (3): Dropout(p=0.25, inplace=False)
    )
  )
  (rnn_stack): Sequential(
    (gru1): BatchRNN(
      (rnn): GRU(32, 64, bias=False, batch_first=True, bidirectional=True)
      (batch_norm_op): SequenceWise (
      BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
      (dropout_op): Dropout(p=0.25, inplace=False)
    )
    (gru2): BatchRNN(
      (rnn): GR

## Load data

In [2]:
import numpy as np
from utils_lib.data_loader import EMDataset, GazeDataLoader
from utils_lib.etdata import get_px2deg

fpath = 'etdata/tobii/Kreuze_Random Recording1_short.npy'
# fpath = 'etdata/lund2013_npy/TH34_img_Europe_labelled_MN.npy'

fdir, fname = os.path.split(os.path.splitext(fpath)[0])

# # load geometry
# geom = {
#     'screen_width': 16,
#     'screen_height': 9,
#     'display_width_pix': 1920,
#     'display_height_pix': 1080,
#     'eye_distance': 60,
# }

# load trial data
X_test = np.load(fpath)

_status = np.isnan(X_test['x']) | \
    np.isnan(X_test['y']) | \
    ~np.in1d(X_test['evt'], config['events'])
X_test['status'] = ~_status
test_dataset = EMDataset(config=config, gaze_data=[X_test])
n_samples = len(test_dataset)



In [3]:
from utils_lib.ETeval import run_infer
import copy
import pandas as pd

kwargs = {
    'cuda': False,
    'use_tqdm': False,
    'eval': False,
}

# load test and run inference
test_loader = GazeDataLoader(
    test_dataset, batch_size=config['batch_size'], num_workers=0, shuffle=False)
_gt, _pr, pr_raw = run_infer(model, n_samples, test_loader, **kwargs)


#glue back the predictions
_data_pr = copy.deepcopy(test_dataset.data)
for _d, _pred in zip(_data_pr, pr_raw):
    _d['evt'] = 0
    _d['evt'][1:] = np.argmax(_pred, axis=1)+1
_data_pr = pd.concat([pd.DataFrame(_d) for _d in _data_pr]).reset_index(drop=True)
_data = pd.DataFrame(X_test)
_data = _data.merge(_data_pr, on='t', suffixes=('', '_pred'), how='left')
_data['evt'] = _data['evt_pred'].replace({np.nan:0})



#save
etdata_pr = ETData()
etdata_pr.load(_data[['t', 'x', 'y', 'status', 'evt']].values, **{'source':'np_array'})

# # convert back to kartesian coordinates
# X_test['x'] = X_test['x'] * px2deg + geom['display_width_pix']/2
# X_test['y'] = X_test['y'] * px2deg + geom['display_height_pix']/2

etdata_pr.plot(show=True, save=False)

AttributeError: 'gazeNET' object has no attribute 'no_grad'

In [ ]:
# # %% plot
# import matplotlib.pyplot as plt
# import utils_lib.etdata as etdata

# # plot raw data
# plt.figure()
# plt.plot(X_test['x'], X_test['y'], '.')

# for e, c in ETData.evt_color_map.items():
#     mask = X_test['evt'] == e
#     plt.plot(X_test['x'][mask], X_test['y'][mask], '.', color=c)